In [ ]:
import pathlib
from importlib import reload

from sequin import sequin

reload(sequin)

cl = sequin.record()

In [ ]:
cloning = pathlib.Path("./Cloning")

cl.add_oligos(cloning / "oligos.tsv")
cl.add_gb(cloning / "Sequences/U00096v3.gb", name="MG1655")

In [ ]:
output_features = []
for feature in cl.sequences["MG1655"].features:
    if "gene" in feature.qualifiers:
        feature.qualifiers["label"] = feature.qualifiers["gene"]
    if feature.type != "gene":
        output_features.append(feature)
cl.sequences["MG1655"].features = output_features

# rne Temperature-sensitive Alleles
* rne-1 (G66S): C1144172T (GGT becomes AGT)
* rne-3071 (L68F): G1144166A (CTC becomes TTC)

In [ ]:
mutation = "C1144172T"
position = int(mutation[1:-1])

In [ ]:
# Find coordinates of the gene
rne_gene = cl.find_features("MG1655", "rne", plot=False)

In [ ]:
cl.show_sequence(
    "MG1655",
    (position - 40, position + 40),
    highlight=(position - 1, position),
    translate=rne_gene,
    strand=-1,
)

In [ ]:
cl.point_mutation("MG1655", mutation, name="MG_rne")
cl.show_sequence(
    "MG_rne",
    (position - 40, position + 40),
    highlight=(position - 1, position),
    translate=rne_gene,
    strand=-1,
)

## Mutagenic oligo

In [ ]:
# Find lagging strand
ori_position = 3925743
terc_position = 1607181

In [ ]:
# Find the potential homology site
binding_site = cl.sequences["MG_rne"][position - 45 : position + 45]

# Make sure it's complementary to the lagging strand
if terc_position < position < ori_position:
    oseq = str(binding_site.seq)
else:
    oseq = str(binding_site.seq.reverse_complement())

# Add 2 phosphorothioate bonds in 5' to escape nucleases
oligo = oseq[0] + "*" + oseq[1] + "*" + oseq[2:]

# Create the mutagenic oligo
cl.new_oligos(oligo, "DE")

## Screening and sequencing

In [ ]:
# Little hack to generate mutation-screening primers
left_padding = 200
right_padding = 350

seq_f, oli_r = cl.make_primers(
    "MG_rne", (position - left_padding, position), plot=False, tm=50
)
oli_f, seq_r = cl.make_primers(
    "MG_rne", (position - 1, position + right_padding), plot=False, tm=50
)
screen_r = oli_f.reverse_complement()
screen_f = oli_r.reverse_complement()

In [ ]:
# Make new oligos and store them in a dict variable
rne_screening = cl.new_oligos([seq_f, screen_r, screen_f, seq_r], "DE")

In [ ]:
# Check specificity
print(cl.find_primers("MG1655", oligos=rne_screening, plot=False))
print()
print(cl.find_primers("MG_rne", oligos=rne_screening, plot=False))

In [ ]:
# Simulate the PCRs
print("# Screening")
cl.pcr("MG_rne", "DE2", "DE3", lim=18)
cl.pcr("MG_rne", "DE4", "DE5", lim=18)
print("\n# Sequencing")
cl.pcr("MG_rne", "DE2", "DE5")

In [ ]:
# Counter-screening oligos
seq_f, oli_r = cl.make_primers(
    "MG1655", (position - left_padding, position), plot=False
)
oli_f, seq_r = cl.make_primers(
    "MG1655", (position - 1, position + right_padding), plot=False
)
screen_r = oli_f.reverse_complement()
screen_f = oli_r.reverse_complement()

# Make new oligos and store them in a dict variable
screening = cl.new_oligos([screen_r, screen_f], "DE")

# Check specificity
print()
print(cl.find_primers("MG1655", oligos=screening, plot=False))
print()
print(cl.find_primers("MG_rne", oligos=screening, plot=False))

In [ ]:
# Simulate the PCRs
cl.pcr("MG1655", "DE2", "DE6", lim=18)
cl.pcr("MG1655", "DE7", "DE5", lim=18)